<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/augmentation/documented_augmentation/GMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install textattack

In [ ]:
import random
import timeit

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

dataset = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/GMB/GMB_dataset.txt', sep="\t", encoding="latin1", index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Per interpretare i POS si consiglia di tener presente:
https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [ ]:
print(f'Contains {dataset["Sentence #"].iloc[-1]} sentences')
print(f'Contains {len(dataset)} words')
dataset.head()

Contains 2999.0 sentences
Contains 66161 words


,Sentence #,Word,POS,Tag
0,1.0,Thousands,NNS,O
1,1.0,of,IN,O
2,1.0,demonstrators,NNS,O
3,1.0,have,VBP,O
4,1.0,marched,VBN,O


In [ ]:
def get_sentence(data):
      agg_func = lambda s: [w for w in s['Word'].values.tolist()]
      grouped = data.groupby('Sentence #').apply(agg_func)
      return [' '.join(s) for s in grouped]

def get_sentence_mantain_struct(data):
      agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                         s["POS"].values.tolist(),
                                                         s["Tag"].values.tolist())]
      return data.groupby('Sentence #').apply(agg_func)

##Rimozione Dell'unico duplicato

In [ ]:
aug = dataset.copy()
sentences = get_sentence(aug)
temp = pd.DataFrame(sentences)
dropped = temp.drop_duplicates(subset = 0, keep = "last")
print(f'{len(dropped)}')
print('Only one sample is duplicated.')
for i, (ag, dr) in enumerate(zip(sentences, dropped.values)):
  if ag != dr:
    remove_index = i+1
    break
remove_index = aug.loc[aug['Sentence #'] == remove_index].index
aug.drop(remove_index, inplace=True)

##Augmentation Phase

###WordNet

In [ ]:
from textattack.augmentation import WordNetAugmenter
random.seed(33)

my_items = []
num_tras_to_add = 2
num_sentences = aug["Sentence #"].iloc[-1]

start_time = timeit.default_timer()
augmenter = WordNetAugmenter(pct_words_to_swap=1,
                               transformations_per_example=num_tras_to_add)
sentences_num = aug.drop_duplicates('Sentence #')['Sentence #'].values
for i, row in enumerate(get_sentence(aug)[0:10]):
  word_comp = aug.loc[aug['Sentence #'] == sentences_num[i], ['POS', 'Tag']]
  word_index = word_comp.index
  result = augmenter.augment(row)
  for aug_row in result:
    num_sentences += 1
    for j, word in enumerate(aug_row.split(' ')):
      my_items.append({'Sentence #': num_sentences,
                       'Word': word,
                       'POS': word_comp['POS'][word_index[j]],
                       'Tag': word_comp['Tag'][word_index[j]]})


elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=aug.columns)
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=4.026953916998536
DIMENSIONE:(442, 4)


In [ ]:
augmeted_new

,Sentence #,Word,POS,Tag
0,3000.0,G,NNS,O
1,3000.0,of,IN,O
2,3000.0,demonstrator,NNS,O
3,3000.0,have,VBP,O
4,3000.0,edge,VBN,O
...,...,...,...,...
437,3019.0,surveillance,NN,O
438,3019.0,scheme,NN,O
439,3019.0,commence,VBZ,O
440,3019.0,operate,VBG,O


###WordEmbedding

In [ ]:
from textattack.augmentation import EmbeddingAugmenter
random.seed(33)

my_items = []
num_tras_to_add = 2
num_sentences = aug["Sentence #"].iloc[-1]

start_time = timeit.default_timer()
augmenter = EmbeddingAugmenter(pct_words_to_swap=1,
                               transformations_per_example=num_tras_to_add)
sentences_num = aug.drop_duplicates('Sentence #')['Sentence #'].values
for i, row in enumerate(get_sentence(aug)[0:10]):
  word_comp = aug.loc[aug['Sentence #'] == sentences_num[i], ['POS', 'Tag']]
  word_index = word_comp.index
  result = augmenter.augment(row)
  for aug_row in result:
    num_sentences += 1
    for j, word in enumerate(aug_row.split(' ')):
      my_items.append({'Sentence #': num_sentences,
                       'Word': word,
                       'POS': word_comp['POS'][word_index[j]],
                       'Tag': word_comp['Tag'][word_index[j]]})


elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=aug.columns)
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=14.007430034000208
DIMENSIONE:(442, 4)


In [ ]:
augmeted_new

,Sentence #,Word,POS,Tag
0,3000.0,Thousand,NNS,O
1,3000.0,of,IN,O
2,3000.0,demonstrations,NNS,O
3,3000.0,have,VBP,O
4,3000.0,paraded,VBN,O
...,...,...,...,...
437,3019.0,monitoring,NN,O
438,3019.0,system,NN,O
439,3019.0,initiates,VBZ,O
440,3019.0,operational,VBG,O


##Final comments

Eseguendo le modifiche basate solo sulla sostituzione di parole tramite sinonimi non ci si aspetta un miglioramento netto, ma comunque presente.
Per ulteriori miglioramenti si tenga conto di:
https://github.com/daixiangau/coling2020-data-augmentation.

Come modello qui ci sono suggerimenti:
https://medium.com/@kaushik.sairam/named-entity-recognition-ner-on-groningen-meaning-bank-gmb-corpus-fba8914be26.